**Install requirements**

In [0]:
!pip3 install 'keras'

**Import libraries**

In [0]:
import os
import sys
import shutil
from random import shuffle

import keras

from sklearn.model_selection import train_test_split

In [0]:
shutil.rmtree('test') 

**Utility Functions**

In [0]:
def create_model():
  model = Sequential()
  model.add(BatchNormalization(input_shape=(224, 224, 3)))
  model.add(Conv2D(filters=16, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
  model.add(MaxPooling2D(pool_size=2))
  model.add(BatchNormalization())

  model.add(Conv2D(filters=32, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
  model.add(MaxPooling2D(pool_size=2))
  model.add(BatchNormalization())

  model.add(Conv2D(filters=64, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
  model.add(MaxPooling2D(pool_size=2))
  model.add(BatchNormalization())

  model.add(Conv2D(filters=128, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
  model.add(MaxPooling2D(pool_size=2))
  model.add(BatchNormalization())

  model.add(Conv2D(filters=256, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
  model.add(MaxPooling2D(pool_size=2))
  model.add(BatchNormalization())

  model.add(GlobalAveragePooling2D())

  model.add(Dense(133, activation='softmax'))

  model.summary()

  return model

def compile_model(model, _opt='adam', _loss='categorical_crossentropy', _metrics=['accuracy']):
  model.compile(_opt, _loss, _metrics)
  return model

def set_checkpointer(_filePath):
  checkpointer = ModelCheckpoint(filepath=_filePath, verbose=1, 
                                 save_best_only=True)
  return checkpointer

def train(model, num_epochs, batch_size, step_size, train_data, train_target, valid_data, valid_target, checkpointer):
  model.fit_generator(datagen.flow(train_data, train_target, batch_size=batch_size),
                    validation_data=(valid_data, valid_target), 
                    steps_per_epoch=train_data.shape[0] // batch_size,
                    epochs=epochs, callbacks=[checkpointer], verbose=1)

# def load_best_model(model, _filePath):
#   model.load_weights(_filePath)

def test(model, test_data, test_target):
  dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_data]

  test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_target, axis=1))/len(dog_breed_predictions)
  print('Test accuracy: %.4f%%' % test_accuracy)

**Dataset preparation**

In [0]:
# Clone github repository with data
if not os.path.isdir('./mlai/Images'):
  !git clone https://github.com/jmagdeska/mlai.git

DATA_DIR = 'mlai/Images'

if not os.path.isdir('train'):
  os.mkdir('train')
if not os.path.isdir('valid'):
  os.mkdir('valid')
if not os.path.isdir('test'):
  os.mkdir('test')

for path, dirs, files in os.walk(DATA_DIR):
  dirs.sort(key = lambda x: x.lower())
  num_samples = len(files)
  i = 0
  
  l = (int)(0.8*num_samples)
  if l != 0:
    train_len = int(0.8*l)
    valid_len = l - train_len
    test_len = num_samples - l
    print(train_len, valid_len, test_len)

    label = path.split("/")[2]
    shuffle(files)

    for filename in files: 
      full_path = os.path.join(path, filename)       
      if i < train_len:        
        dir_name = os.path.join('train', label)
        if not os.path.isdir(dir_name):
          os.mkdir(os.path.join("train", label))
        shutil.move(full_path, dir_name)
      elif i < (train_len + valid_len):
        dir_name = os.path.join('valid', label)
        if not os.path.isdir(dir_name):
          os.mkdir(os.path.join("valid", label))
        shutil.move(full_path, dir_name)
      else:
        dir_name = os.path.join('test', label)
        if not os.path.isdir(dir_name):
          os.mkdir(os.path.join("test", label))
        shutil.move(full_path, dir_name)

      i += 1
  